# 1.1 隐式基类 - Object 
每个python 定义的类都会隐式的继承object 类 

In [1]:
class X:
    pass 
print(X.__class__)
print(X.__class__.__base__)

<class 'type'>
<class 'object'>


上述输出说明： 类其实是type类的一个对象， type类的基类为object 类

# 1.2 基类中的__init__()方法 
对象的生命周期主要包括创建、初始化、销毁   
继承自object类的子类，总是可以对它的属性进行拓展   
例如下面这个例子不需要对width和height 进行初始化

In [2]:
class Rectangle:
    def area(self):
        return self.width * self.height  # 这种方式在python中是合法的，但不建议使用

r = Rectangle()
r.width, r.height = 2, 3 # 在使用时赋值
r.area()

6

延迟初始化属性的设计虽然具有一定的灵活性，但从长远来看，这是一种糟糕的设计 

In [3]:
import this 

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


Explicit is better than implict 显示而非隐式，对于每个__init__都应当显示的指定要初始化的变量

### 1.3 基类中实现 __init__方法 

In [4]:
# 多态设计 
class Card:
    def __init__(self, rank, suit):
        """
        rank: 号码 
        suit: 花色
        """
        self.suit = suit 
        self.rank = rank 
        self.hard, self.soft = self._points() # 类内部函数命名以_开始投， 表示该函数可以被继承访问 

class NumberCard(Card):
    def _points(self):
        return int(self.rank), int(self.rank)

class FaceCard(Card):
    def _points(self):
        return 10, 10 

class AceCard(Card):
    def _points(self):
        return 1, 11


> 内部函数以 _ 开头命名，表示该函数可被继承访问    
> 以 __ 开头， 为类内私有函数命名， 该函数不可被继承访问  

- 常见的多态设计， 每个子类为_points 提供特有的实现 
- 所有的子类有相同的方法名和属性名 



In [5]:
# 如果只是简单的去定义牌，可以使用如下方式 
cards = [AceCard('A', '♠'), NumberCard('2', '♠')]

但这样的枚举，需要枚举52张牌，麻烦且容易出错，可以考虑工厂函数   
先看一些其他的方法 

### 1.4 使用__init__创建常量清单  

很多情况下，应用会包括一个常量集合， 静态常量 构成了 **策略(Strategy)** 或 **状态(State)** 的一部分  
python 中并没有提供简单而直接的方式来定义一个不可变对象。  

这个例子中，将花色定义为一个不可变对象是有意义的

In [6]:
class Suit:
    def __init__(self, name, symbol):
        """
        name: 花色名
        symbol: 花色标识
        """
        self.name = name 
        self.symbol = symbol 

Spade, Heart, Club, Diamond = Suit('Spade', '♠'), Suit('Heart', '♥'), Suit('Club', '♣'), Suit('Diamond', '♦')

把创建好的花色对象做缓存，使得在调用时对象可以被重复使用，性能将会得到显著提升 
(python 对象只是概念上的常量，事实上仍然是可变的， 使用额外的代码使得对象完全不可变可能会更好)

### 1.5 通过工厂函数调用__init__ 

实现工厂有两种途径 
- 1. 定义一个函数， 返回不同的类 
- 2. 定义一个类， 包含了创建对象的方法  

第二种方式是完整的工厂设计模式，在类似java的语言中，工厂类层次结构是必须的， 因为*语言本身不支持脱离类而单独存在的函数* (好像还真是)  
在 python 里 类不是必须的， 只有在特别复杂的场景下，工厂类才是不错的选择  
python 的优势之一就是 对于只需要简单定义一个函数就能做到的事没有必要去定义类层次结构  

类定义的优势是 可以通过继承来使得代码被更好的重用  （而不仅仅是封装起来比较好看而已 ）

如果工厂类并没有重用任何代码，那么类层次结构在python中并没有多大用处， 完全可以用函数替代  

如下是一个工厂函数的例子

In [7]:
def card(rank, suit):
    """ 生成对应的card 对象的工厂函数 
    rank: 号码 
    suit: 花色
    """
    if rank == 1: 
        return AceCard('A', suit) 
    elif 2 <= rank <= 10:
        return NumberCard(str(rank), suit) 
    elif 11<= rank <= 13:
        name = {
            11: 'J',
            12: 'Q',
            13: 'K'
        }
        return FaceCard(name[rank], suit)
    else: # 应当尽量避免不明确的分支， 枚举所有场景后通过 else throw exception 是一个更明智的方法 
        raise Exception('Rank out of range')
    
# 可以通过这种方式来构建卡片对象 
deck = [card(rank, suit) for rank in range(1, 14) for suit in (Spade, Heart, Club, Diamond)]

In [8]:
## version 2  使用映射来简化设计 
def card4(rank, suit):
    """
    使用映射来简化上述工厂方法  
    """
    return {
        1: AceCard('A', suit),
        11: FaceCard('J', suit),
        12: FaceCard('Q', suit),
        13: FaceCard('K', suit)
    }.get(rank, NumberCard(str(rank), suit))

### v3: 代码重复度进一步减少 
def card5(rank, suit):
    """  映射到一个二元组
    """
    class_, rank_str = {
        1: (AceCard, 'A'),  # 类本身也是一个对象 
        11: (FaceCard, 'J'),
        12: (FaceCard, 'Q'),
        13: (FaceCard, 'K')
    }.get(rank, (NumberCard, str(rank)))
    return class_(rank_str, suit)

deck = [card4(rank, suit) for rank in range(1, 14) for suit in (Spade, Heart, Club, Diamond)]

上述card5 实现由两个缺点 
- 从rank 值 映射到 类对象 很少见   
- 两个参数只有一个用于类的初始化 

从rank 映射到一个相对简单的类或函数对象，而不必提供目的不明确的参数，是一个更好的选择

In [9]:
from functools import partial 
def card6(rank, suit):
    """ 使用partial 函数来进行类对象的生产 
    """
    class_ = {
        1: partial(AceCard, 'A'),
        11: partial(FaceCard, 'J'),
        12: partial(FaceCard, 'Q'),
        13: partial(FaceCard, 'K')
    }.get(rank, partial(NumberCard, str(rank)))
    return class_(suit)

# 重写花色类方法的生成 
def suit2(symbol):
    """ 花色类重写 """
    class_ = {
        '♠': partial(Suit, 'Spade'),
        '♥': partial(Suit, 'Heart'),
        '♣': partial(Suit, 'Club'),
        '♦': partial(Suit, 'Diamond')
    }.get(symbol)

    return class_(symbol)


partial() 函数的使用在函数式编程中很常见 

### 1.6 在每个子类中实现__init__()方法  
考虑重构card类，并将部分初始化在card类中实现 

In [10]:
class Card:
    def __init__(self, rank, suit, hard, soft):
        """
        rank: 号码 
        suit: 花色 
        hard: 
        soft
        """
        self.rank = rank 
        self.suit = suit 
        self.hard = hard 
        self.soft = soft 

class AceCard(Card):
    def __init__(self, rank, suit):
        super().__init__('A', suit, 1, 11)
    
class NumberCard(Card):
    def __init__(self, rank, suit):
        super().__init__(str(rank), suit, int(rank), int(rank))

class FaceCard(Card):
    def __init__(self, rank, suit):
        super().__init__({11: 'J', 12: 'Q', 13: 'K'}.get(int(rank)), suit, 10, 10)

经过上述重写以后，工厂函数的实现也会变得简洁 
> 事实上， 在__init__ 函数和 工厂函数之间有些权衡，通常直接调用 比 __init___并把构造复杂性分发给工厂函数 更好， 当需要封装复杂的构造逻辑时，工厂函数才是一个更好的选择 

In [11]:
def card10(rank, suit):
    """ 工厂方法v10 """
    if rank == 1: return AceCard(rank, suit) 
    elif 2 <= rank < 11: return NumberCard(rank, suit) 
    elif 11 <= rank < 14: return FaceCard(rank, suit) 
    else: raise Exception('rank is invalid ')
    

### 1.7 简单的组合对象 
一个组合对象也可以被称为**容器** 
> 面向对象的设计原则 
> 单一功能原则 : 对象应该仅具有一种单一功能   
> 开闭原则： 对于拓展开发，对于修改封闭    
> 里氏替换原则： 程序中的对象应该是在可以不改变程序正确性的前提下被它的子类所替换   
> 接口隔离原则： 多个特定的客户端接口要好于一个用途宽泛的接口    
> 依赖反转原则： 一个方法应该遵守依赖于抽象而不是一个实例原则    

类定义的一个优势： 类给对象提供了简单的、不需要实现的接口 （然而对于python来说，却不是必须的，可能这就是脚本语言的特性吧）

设计一个集合类，通常有一下三种方法：  
- 封装： 这个设计是基于现有集合类来定义一个新类，属于外观模式的一个使用场景 
- 扩展： 对现有集合类进行拓展， 通常使用定义子类的方式来实现  
- 创建： 重新设计 

In [12]:
"""
如下是一个封装的例子， 内部事实上是对 list 的一个封装， 调用方法也都是list 的方法
"""
import random 
class Deck:
    def __init__(self):
        # 牌堆
        self._cards = [card10(rank, suit) for rank in range(1, 14) for suit in [Spade, Heart, Club, Diamond]]
        random.shuffle(self._cards) 

    def pop(self):
        """ 发牌 """
        return self._cards.pop()
    

一般来说， 外观模式或者封装类中的方法实现只是对底层对象相应函数的代理调用（有点多余?）

In [13]:
"""
拓展集合类, 好处是不用重新实现Pop方法, 只需继承即可
"""
class Deck(list):
    def __init__(self):
        _cards = [card10(rank, suit) for rank in range(1, 14) for suit in [Spade, Heart, Club, Diamond]]
        super().__init__(_cards)
        random.shuffle(self) #  牛啊？self 还可以这么用 ？ 

在一些场景下，在子类中需要显示的调用基类的函数来完成适当的实现

In [15]:
"""
以下是一个可以适应更多需求的设计(模拟发牌机，可以有多副牌)
"""
class Deck3(list):
    def __init__(self, decks=1):
        super().__init__() 
        for i in range(decks):
            self.extend([card10(rank, suit) for rank in range(1, 14) for suit in [Spade, Heart, Club, Diamond]])
        
        random.shuffle(self)
        burn = random.randint(1, 52)
        for i in range(burn): self.pop() 

### 1.8 复合的组合对象  

定义Hand类，用来模拟打牌策略 

In [16]:
class Hand:
    def __init__(self, dealer_card):
        self.dealer_card = dealer_card 
        self.cards = [] 

    def hard_total(self):
        """ 手牌hard点数 """
        return sum(c.hard for c in self.cards) 
    
    def soft_total(self):
        """ 手牌soft点数"""
        return sum(c.soft for c in self.cards)


In [17]:
# 以下是初始化一个Hand对象，事实上非常麻烦
d = Deck() 
h = Hand(d.pop())
h.cards.append(h.cards.append(d.pop())) 

In [23]:
h.cards[0].suit.symbol

'♥'

\__init__()方法通常应该返回一个完整的对象，但对象的构造可能是复合的。 
通常考虑使用一个流畅接口来完成逐个将对象添加到集合，同时将集合对象作为构造函数的参数来完成初始化 

In [26]:
class Hand2:
    # * 号运算符为解包操作 
    def __init__(self, dealer_card, *cards):
        """ 
        dealer_card: 新发的牌
        cards: 手牌 
        """
        self.dealer_card = dealer_card 
        self.cards = list(cards)
    
    def hard_total(self):
        return sum(c.hard for c in self.cards)
    
    def soft_total(self):
        return sum(c.soft for c in self.cards)
    
d = Deck() 
h = Hand2(d.pop(), d.pop(), d.pop())
h.cards

[<__main__.FaceCard at 0x21dd37d6940>, <__main__.NumberCard at 0x21dd37d6a30>]

### 1.9 不带__init__()方法的无状态对象 

对于策略模式的对象来说，这是常见的设计。一个策略对象以插件的形式复合在主对象上来完成一种算法或逻辑  
策略类通常和享元模式一起使用： 在策略对象中避免内部存储。所有需要的值都从策略对象的方法参数传入。 策略对象本省是无状态的，是一系列函数的集合  
在策略对象中，避免内部存储 
> 策略模式：
> - 上下文Context 维护指向具体策略的引用， 且仅通过策略接口与该对象进行交流 
> - 策略Strategy 接口是所有具体策略的通用接口，声明一个上下文用于执行策略的方法 
> - 具体策略 Concrete Strategy 实现了上下文所有不同算法， 当上下文需要计算的时候，会调用对应的execute方法  
> - 客户端 Client 会创建一个特定策略对象并将其传递给上下文， 上下文会提供一个设置器以方便客户端在运行时替换相关联的策略 
> ![Alt text](image-1.png)    

> 享元模式： 结构型设计模式，摒弃了在所有对象中保存数据的模式，在有限内存中可以存储更多对象    
> - 享元模式是一种优化    
>![Alt text](image-2.png)

In [30]:
"""
21 点 策略类 (当前阻断的架构也可以用这种方式来写，避免每次大量的对象构造和重复工作) 
供player 实例提供游戏模式选择 
"""

class GameStrategy:
    def insurance(self, hand):
        """ 保险 """
        return False 
    
    def split(self, hand):
        """ 分牌 """
        return False
     
    def double(self, hand):
        """ 双倍 """
        return False 
    
    def hit(self, hand):
        """ 叫牌 """
        return sum(c.hard for c in hand.cards) <= 17
    
dump = GameStrategy() # 单例策略对象

### 1.10 一些其他类的定义  
Table 类， 模拟器类， 用于追踪与players 集合所有相关操作的状态  

In [31]:
class Table:
    def __init__(self):
        """ 初始化牌局 """
        self.deck = Deck() 

    def place_bet(self, amount):
        """ 下注 """
        print('Bet', amount) 

    def get_hand(self):
        """ 分牌 """
        try:
            self.hand = Hand2(d.pop(), d.pop(), d.pop())
            self.hole_card = d.pop() 
        except IndexError:
            # 牌堆用完，需要重新洗牌 
            self.deck = Deck() 
            return self.get_hand()
        
        print ('Deal', self.hand)
        return self.hand
    
    def can_insure(self, hand):
        """ 下注 """
        return hand.deader_card.insure

python 的 abc 类定 可以用来创建抽象基类 (Abstract Base Classes)

In [32]:
# 不使用abc 创建抽象基类，通过在基类函数中抛出异常来实现 
class BettingStrategy:
    def bet(self):
        raise NotImplementedError('No bet method') 
    
    def record_win(self):
        pass 

    def record_loss(self):
        pass 

class Flat(BettingStrategy):
    def bet(self):
        return 1

In [33]:
""" 
使用abc 来创建 抽象基类
"""
import abc 
from abc import ABC, ABCMeta, abstractmethod 

class BettingStrategy2(ABCMeta):
    
    @abstractmethod 
    def bet(self):
        return 1 
    
    def record_win(self):
        pass 

    def record_loss(self):
        pass 
    

使用这种方法有两种好处   
- 阻止了对抽象基类BettingStrategy2 的实例化 
- 任何没有提供bet 方法的子类无法被实例化 

如果抽象基类实现了对bet 方法， 那么就是合法的，且可以通过 super().bet() 来调用

### 1.11 多策略的__init__()方法 

备忘录模式 ：行为设计模式， 允许在不暴露实现细节的情况下保存和恢复对象之前的状态 
> ![Alt text](image.png)

In [37]:
"""
两种Hand类的实现方法 
"""
class Hand3:
    def __init__(self, *args, **kw):
        if len(args) == 1 and isinstance(args[0], Hand3):
            # Clone an existing hand; offen a bad idead 
            print('existing hand ')
            other = args[0]
            self.dealer_card = other.dealer_card 
            self.cards = other.cards 
        else:
            # build a new hand, 
            dealer_card, *cards = args 
            self.dealer_card = dealer_card 
            self.cards = list(cards)

# 基于一个已有的对象，创建一个Hand对象， 可以用来创建一个Hand对象的备忘录模式 
h = Hand3(deck.pop(), deck.pop(), deck.pop())
memento = Hand3(h) 

existing hand 


In [62]:
"""
多种策略用于初始化 , 以下这种方式需要大量的文档来说明初始化逻辑 
"""
class Hand4:
    def __init__(self, *args, **kw):
        if len(args) == 1 and isinstance(args[0], Hand4):
            # Clone an existing hand 
            other = args[0]
            self.dealer_card = other.dealer_card
            self.cards = other.cards 
        elif len(args) == 2 and isinstance(args[0], Hand4) and 'split' in kw:
            # 玩家选择分牌
            other, card = args  # other 为原来的 手牌， cards 为新牌 
            self.dealer_card = other.dealer_card 
            self.cards = [other.cards[kw['split']], card]
        elif len(args) == 3:
            # build a new fresh hand
            dealer_card, *cards = args
            self.dealer_card = dealer_card 
            self.cards = list(cards)
        else:
            raise TypeError('Invalid constructor args={0!r} kw={1!r}'.format(args, kw))  # 0!r 表示将第一个参数表示为字符串形式
        
    def __str__(self):
        return ','.join(map(str, self.cards))
    
d = Deck() 
h = Hand4(d.pop(), d.pop(), d.pop()) 
print([card.rank for card in h.cards], h.dealer_card.rank)
s1 = Hand4(h, d.pop(), split=0)
print([card.rank for card in s1.cards], s1.dealer_card.rank)
s2 = Hand4(h, d.pop(), split=1)
print([card.rank for card in s2.cards], s2.dealer_card.rank)

# s2 = Hand4(h, d.pop(), split=1)

['A', '2'] 2
['A', '9'] 2
['2', '10'] 2


In [65]:
"""
使用静态函数作为代理函数, 在代码结构上将更加具有优势 
"""
class Hand5:
    def __init__(self, dealer_card, *cards):
        """ 
        dealer_card: 庄家的明牌 
        cards: 闲家的手牌
        """
        self.dealer_card = dealer_card 
        self.cards = list(cards) 
    
    @staticmethod
    def freeze(other):
        """ 备忘录模式 """
        hand = Hand5(other.dealer_card, *other.cards)
        return hand

    @staticmethod
    def split(other, card0, card1):
        """ 分牌 """
        hand0 = Hand5(other.dealer_card, other.cards[0], card0)
        hand1 = Hand5(other.dealer_card, other.cards[1], card1)
        return hand0, hand1

    def __str__(self):
        return ",".join(map(str, self.cards))
    

d = Deck() 
h = Hand5(d.pop(), d.pop(), d.pop())
s1, s2 = Hand5.split(h, d.pop(), d.pop())


尽管使用静态方法作为代理相对于V1 直接使用__init__() 方法已经简化了很多，但是违背了一个原则：   
**使用已有的set对象来建立frozenset对象** 

### 1.12 更多的__init__()技术 
以建立 Player 类为例    
> 突然发想，本章的代码应该是要在对游戏进行号抽象的情况下进行的，应该首先就有了一个比较详细的文档  
> 而这样的抽象也是有很高的技术含量的，后续需要增强这方面的能力  

In [66]:
class Player:
    def __init__(self, table, bet_strategy, game_strategy):
        """ 
        table: 模拟牌桌  
        bet_strategy: 下注策略 
        game_strateyg： 游戏策略 
        """
        self.table = table 
        self.bet_strategy = bet_strategy 
        self.game_strategy = game_strategy 

    def game(self):
        # 下注 
        self.table.place_bet(self.bet_strategy.bet()) 
        
        # 发牌
        self.table.get_hand()

        # 是否可以买保险 
        if self.table.can_insurance(self.hand):
            if self.game_strategy.insurance(self.hand):
                self.table.insure(self.bet_strategy.bet())

        # 后续补充更多的逻辑 



上述实现中__init__()函数只是进行了对象保存的功能，如果有很多的参数，那么复制将会变得非常臃肿   
可以通过把关键字参数直接转换为内部变量，以提供一个非常短且灵活的初始化方式  

In [68]:
class Player2:
    def __init__(self, **args):
        """ 必须提供 table, bet_strategy, game_strategy 对象="""
        self.__dict__.update(args)  # __dict__ 是一个特殊的属性，用于存储一个对象的可变属性  

    def game(self):
        # 下注 
        self.table.place_bet(self.bet_strategy.bet()) 
        
        # 发牌
        self.table.get_hand()

        # 是否可以买保险 
        if self.table.can_insurance(self.hand):
            if self.game_strategy.insurance(self.hand):
                self.table.insure(self.bet_strategy.bet())

        # 后续补充更多的逻辑 

table = Table()
flat_bet = Flat()
dump = GameStrategy()
p1 = Player2(table=table, bet_strategy=flat_bet, game_strategy=dump)


这种方式牺牲了大量的可读性来换来代码的简洁性， 需要再初始化时提供对应的变量名   
这种方式需要避免参数名的冲突，而 **kw 提供了很少的信息  
在设计之初， 应当考虑设计一些灵活的子类，而不是完美的基类  

In [70]:
"""
必须参数设置为位置参数 
可选参数设置为关键字参数 
"""
class Player3:
    def __init__(self, table, bet_strategy, game_strategy, **extras):
        self.table = table 
        self.bet_strategy = bet_strategy 
        self.game_strategy = game_strategy 
        self.__dict__.update(extras)

#### 1.12.1 带有类型验证的初始化  

类型验证是为了验证所有的参数是恰当的类型。 

In [71]:
""" 以下的实现可能会带来问题 : 不得不创建子类 """

class ValidPlayer:
    def __init__(self, table, bet_strategy, game_strategy):
        assert isinstance(table, Table) 
        assert isinstance(bet_strategy, BettingStrategy) 
        assert isinstance(game_strategy, GameStrategy)

        self.bet_strategy = bet_strategy 
        self.game_strategy = game_strategy 
        self.table = table 

**为了使工作量最小化，无论如何我们都必须提供文档、日志和测试用例**

In [72]:
class Player:
    def __init__(self, table, bet_strategy, game_strategy):
        """
        create a new palyer associated with a table, and configured  with proper betting and game strategies 
        :param table: an instance of class: Table 
        :param bet_strategy: an instance of class: BettingStrategy 
        :param game_strategy: an instance of class: GameStrategy 
        """
        self.table = table 
        self.bet_strategy = bet_strategy 
        self.game_strategy = game_strategy 
        
    

命名规范：  
- 大部分名称是公有的  
- 以 _ 开头的名字通常不完全公有， 这些函数通常是实现细节 
- 以 __ 开头的是 python 内部的，程序中一般不应该使用  


通常接口函数会有说明文档和测试用例， 而实现细节的函数只需提供简单的说明即可  